In [11]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [12]:
%matplotlib inline
plt.style.use('ggplot')
# font = {'weight': 'bold',
#         'size':   16}
# plt.rc('font', **font)

In [3]:
df_courses = pd.read_csv('data/courses.csv')

In [4]:
df_assessments = pd.read_csv('data/assessments.csv')

In [5]:
df_vle = pd.read_csv('data/vle.csv')

In [6]:
df_studentInfo = pd.read_csv('data/studentInfo.csv')

In [7]:
df_studentRegistration = pd.read_csv('data/studentRegistration.csv')

In [8]:
df_studentAssessment = pd.read_csv('data/studentAssessment.csv')

In [9]:
df_studentVle = pd.read_csv('data/studentVle.csv')

## Initial Approach
Since the schema contains 7 data frames, I want to look into each of them one-by-one to get a better understanding of them to then be able to make hypothesis and generate ideas of what data frame I need to build ahead of any modeling. I also want to hone in on the dependent variable(s) I want to end up creating prediction models for.

I've listed each raw Data Frame below and the questions I had after reading the documentation (https://analyse.kmi.open.ac.uk/open_dataset)

### 1 - Courses 
* How many modules are there?
* How many presentations per module?
* What is the relationship b/w modules and presentations
* What the average length of each presentation? Is it unique to the presentation or to the module?
* What is the detail behind the CCC, EEE, and GGG modules they mention in the description

### 2 - Assessments
* How many assessments are there per presentation?
* Are there always the same assessments per module?
* Does the Date - i.e. number of days from the start of the module-presentation till the final presentation date tell us the length of each module-presentation?
* What weights add up to 100% per each module-presentation? Is the Final Exam 100% and then TMA+CMA = 100%?

### 3 - VLE
* Does the number of total VLE belong to a presentation or to a module?
* How many different types of VLEs are there per module-presentation?
* What is the **activity type** field contain. Descripton says it's the, "role associated with the module materials."

### 4 - studentInfo
* Do students have multiple rows in this DF depending on each module-presentation they've participated in?
* How many Final Scores are there per student?
* Is there any correlation between the 6 demographic vars (gender, region, highest_education, imd_band, age_band, disability) and Final Scores?
* Is there any correlation b/w the num_previous_attempts and studied_credits and Final Scores?
* Could there be any leakage in the data with students who previously took the module-presentation?

### 5 - studentRegistration
* What is the avg number of days to register per student?
* How many students unregistered from a module-presentation?
* What is the % of registered/Total Registrations and also unregistered/Total Registrations?

### 6 - studentAssessment
* How many on-time vs. late submissions are there?
* What is the average score split by on-time and late submissions?
* What % of assessments are "banked" i.e. if the assessment result was transferred from a previous presentation/
* What pct is Num Fails (<40 Score) to total Assessments

### 7 - studentVle
* Can students have multiple rows per a singular Vle due to tracking their interaction with it by day?
* How many clicks are there per Student per VLE
* How many clicks per student per day are there per VLE
* Plot Usage vs. Date